In [1]:
#https://www.youtube.com/watch?v=k_VAKyzggJI
import nltk
import inflect
import contractions
from bs4 import BeautifulSoup
import re, string, unicodedata
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import re
import numpy as np
from collections import Counter
from nltk import ngrams
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [9]:
df = pd.read_csv("data_clean.csv")
df

,text,label
0,As more patients choose dental implants as the...,0
1,Diabetes mellitus (DM) is a recognized risk fa...,0
2,Risk factors associated with tooth loss have b...,0
3,To retrospectively assess clinical and radiogr...,0
4,Periodontal disease often develops in patients...,0
...,...,...
583,The journey of gastric phytobezoar followed by...,1
584,Determinants of protein-energy malnutrition in...,1
585,Prosthetic rehabilitation of an edentulous pat...,1
586,T-bar clasp-retained removable partial denture...,1


In [10]:
def cleansing(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'/(?<!\S).(?!\S)\s*/', '', text)
    return text

In [11]:
df['text'] = df['text'].apply(cleansing)
df.head()

,text,label
0,as more patients choose dental implants as the...,0
1,diabetes mellitus dm is a recognized risk fact...,0
2,risk factors associated with tooth loss have b...,0
3,to retrospectively assess clinical and radiogr...,0
4,periodontal disease often develops in patients...,0


In [12]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df

,text,label
0,patients choose dental implants primary treatm...,0
1,diabetes mellitus dm recognized risk factor de...,0
2,risk factors associated tooth loss studied how...,0
3,retrospectively assess clinical radiographic o...,0
4,periodontal disease often develops patients di...,0
...,...,...
583,journey gastric phytobezoar followed tomography,1
584,determinants proteinenergy malnutrition commun...,1
585,prosthetic rehabilitation edentulous patient c...,1
586,tbar claspretained removable partial denture a...,1


In [14]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df['text'] = df.text.apply(lemmatize_text)
df

,text,label
0,patient choose dental implant primary treatmen...,0
1,diabetes mellitus dm recognized risk factor de...,0
2,risk factor associated tooth loss studied howe...,0
3,retrospectively ass clinical radiographic outc...,0
4,periodontal disease often develops patient dia...,0
...,...,...
583,journey gastric phytobezoar followed tomography,1
584,determinant proteinenergy malnutrition communi...,1
585,prosthetic rehabilitation edentulous patient c...,1
586,tbar claspretained removable partial denture a...,1


In [25]:
def perform_oversampling(df):
    df_non = df[df['label'] == 1]
    df_exclude = df[df['label'] == 0]
    df_low_oversampled = df_exclude.sample(df_non.shape[0], replace=True)
    return pd.concat([df_low_oversampled, df_non], axis=0)

print('Original data:')
print(df['label'].value_counts())

df = perform_oversampling(df)

print('-'*10)
print('After oversampling:')
print(df['label'].value_counts())

Original data:
0    540
1     48
Name: label, dtype: int64
----------
After oversampling:
0    48
1    48
Name: label, dtype: int64


In [26]:
text = df['text'].values
labels = df['label'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [27]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(text, encoded_labels, stratify = encoded_labels)

In [28]:

vocab_size = 3000 
oov_tok = ''
embedding_dim = 100
max_length = 200
padding_type='post'
trunc_type='post'


tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [29]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 100)          300000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              84480     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 24)                3096      
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [31]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("acc of test set", accuracy_score(test_labels,pred_labels))

Epoch 1/5
2/2 [==============================] - 0s 150ms/step - loss: 0.6567 - accuracy: 0.8906 - val_loss: 0.6747 - val_accuracy: 0.6250
Epoch 2/5
2/2 [==============================] - 0s 128ms/step - loss: 0.6384 - accuracy: 0.9531 - val_loss: 0.6646 - val_accuracy: 0.7500
Epoch 3/5
2/2 [==============================] - 0s 130ms/step - loss: 0.6118 - accuracy: 1.0000 - val_loss: 0.6507 - val_accuracy: 0.8750
Epoch 4/5
2/2 [==============================] - 0s 130ms/step - loss: 0.5747 - accuracy: 1.0000 - val_loss: 0.6320 - val_accuracy: 0.8750
Epoch 5/5
1/1 [==============================] - 0s 39ms/step
acc of test set 0.8333333333333334


In [34]:
from keras.models import load_model
    
model.save('lstm-final') 

INFO:tensorflow:Assets written to: lstm-final\assets


INFO:tensorflow:Assets written to: lstm-final\assets


In [32]:
y_predict = np.where(model.predict(test_padded)> 0.5,1,0)
print(classification_report(test_labels, y_predict))

1/1 [==============================] - 0s 38ms/step
              precision    recall  f1-score   support

           0       0.79      0.92      0.85        12
           1       0.90      0.75      0.82        12

    accuracy                           0.83        24
   macro avg       0.84      0.83      0.83        24
weighted avg       0.84      0.83      0.83        24



In [ ]:
odel = keras.models.load_model('lstm-final')

In [ ]:
# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)
# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)
# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
for i in range(len(sentence)):
    print(sentence[i])
    if pred_labels[i] == 1:
        s = 'Included'
    else:
        s = 'Excluded'
    print("Predicted Label : ",s)